## Imports 


In [21]:
import pandas as pd
import numpy as np
import time

# Cleaning Company Names

### Long list of Companies

In [22]:
stock_codes = pd.read_csv(r'EOD-datasets-codes.csv')
# stock_codes.head()

In [23]:
stock_codes['Tick'] = stock_codes['Code'].map(lambda x: x[4:])
stock_codes['Name'] = stock_codes['Company Name'].map(lambda x: x[:x.find('(')] )
stock_codes.head(10)

,Code,Company Name,Tick,Name
0,EOD/AAPL,"Apple Inc. (AAPL) Stock Prices, Dividends and ...",AAPL,Apple Inc.
1,EOD/ADES,Advanced Emissions Solutions Inc. (ADES) Stock...,ADES,Advanced Emissions Solutions Inc.
2,EOD/ACST,"Acasti Pharma Inc. (ACST) Stock Prices, Divide...",ACST,Acasti Pharma Inc.
3,EOD/ACTS,Actions Semiconductor Co. Ltd. (ACTS) Stock Pr...,ACTS,Actions Semiconductor Co. Ltd.
4,EOD/ADMP,Adamis Pharmaceuticals Corporation (ADMP) Stoc...,ADMP,Adamis Pharmaceuticals Corporation
5,EOD/ABAX,"ABAXIS Inc. (ABAX) Stock Prices, Dividends and...",ABAX,ABAXIS Inc.
6,EOD/GCH,Aberdeen Greater China Fund Inc. (GCH) Stock P...,GCH,Aberdeen Greater China Fund Inc.
7,EOD/ADMS,Adamas Pharmaceuticals Inc. (ADMS) Stock Price...,ADMS,Adamas Pharmaceuticals Inc.
8,EOD/IAF,Aberdeen Australia Equity Fund Inc (IAF) Stock...,IAF,Aberdeen Australia Equity Fund Inc
9,EOD/ABB,"ABB Ltd (ABB) Stock Prices, Dividends and Splits",ABB,ABB Ltd


In [24]:
scl = stock_codes.columns.tolist()
print(scl)

stock_codes = stock_codes[ ['Tick', 'Name','Code','Company Name']  ]
stock_codes.head()

['Code', 'Company Name', 'Tick', 'Name']


,Tick,Name,Code,Company Name
0,AAPL,Apple Inc.,EOD/AAPL,"Apple Inc. (AAPL) Stock Prices, Dividends and ..."
1,ADES,Advanced Emissions Solutions Inc.,EOD/ADES,Advanced Emissions Solutions Inc. (ADES) Stock...
2,ACST,Acasti Pharma Inc.,EOD/ACST,"Acasti Pharma Inc. (ACST) Stock Prices, Divide..."
3,ACTS,Actions Semiconductor Co. Ltd.,EOD/ACTS,Actions Semiconductor Co. Ltd. (ACTS) Stock Pr...
4,ADMP,Adamis Pharmaceuticals Corporation,EOD/ADMP,Adamis Pharmaceuticals Corporation (ADMP) Stoc...


### Pull FTSE 100 from Wikipedia.

In [25]:
wiki_Ftse = r'https://en.wikipedia.org/wiki/FTSE_100_Index'

FTSE100 = pd.read_html(wiki_Ftse)[2]

FTSE100 = FTSE100.rename(columns= FTSE100.iloc[0])
FTSE100 = FTSE100.iloc[1:]
FTSE100.head()

,Company,Ticker,FTSE sector[9]
1,3i,III,Financial Services
2,Admiral Group,ADM,Nonlife Insurance
3,Anglo American plc,AAL,Mining
4,Antofagasta,ANTO,Mining
5,Ashtead Group,AHT,Support Services


### Create SQL Connection

In [35]:
# Login details: 
DSN_Name = 'traderDSN' ; Login_ID = 'basic' ; pwd = 'pwd1'

# Create a DSN connection and a cursor to do stuff. 
from sqlalchemy import create_engine
import pandas as pd

# The engine does the same job as a connection and a cursor
engine = create_engine(r'mssql+pyodbc://'+Login_ID+':'+pwd+'@'+ DSN_Name) 

# Alpha Vantage API

In [27]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import requests
import json
from pprint import pprint
import time 

### Single Company Example 

In [36]:
# Code to Read a single page
# url = "https://www.alphavantage.co/query"
# function = "TIME_SERIES_DAILY_ADJUSTED"
# symbol = "MSFT"
# api_key = "LSJQLCCGC6A3RJGC"  #  <-- goes here your API KEY

# data = { "function": function, 
#          "symbol": symbol, 
#          "apikey": api_key } 
# page = requests.get(url, params = data).json()

# pprint(page)

In [30]:
def parse_one_ticker(page):
    ''' Creates a dataframe for a single stock ticker. Has sensible names and is ready for import to SQL'''

    ticker  = page['Meta Data']['2. Symbol']
    meta = page['Meta Data']
    #print(ticker,meta)

    # Will fail by here, if the API is giving problems
    all_trades = pd.DataFrame() # reset the data strucutre. 
    for date in page['Time Series (Daily)']:
        trades = page['Time Series (Daily)'][date]
        trades = {key[3:]:value for key,value in trades.items()}
        day_of_trades = pd.DataFrame.from_dict(trades,orient='index').transpose()
        all_trades = all_trades.append(day_of_trades)

    return all_trades

In [31]:
#all_trades = parse_one_ticker(page)
#tablename = 'abcd'
#all_trades.to_sql(tablename, engine, if_exists='append')

### Import a list of Companies

In [32]:
def vantage_scrape_list(companies_df, tablename, schema, engine, function, outputsize, api_key):
    '''  Scrape Tweets from Alpha-Vantage website. Expects a dataframe with column names 'Ticker' and 'Company' '''
    url = "https://www.alphavantage.co/query"
    
    num_firms = companies_df.shape[0]
    delay = 2 # Time to wait after an failed API call.
    min_comps = num_firms # Minimum companies to return.

    failed_codes = [] 
    failed_msg = []
    data_flag = [0] * num_firms
    ticker_flag = [0] * num_firms
    
    nthis, nlast = 0, -1 
    
    # Keep retrying until a certain number are found.
    while sum(data_flag) < min_comps and nlast != nthis:
        for index, company in companies_df.iterrows():
            symbol = company['Ticker'].replace('.','')
            firm_name = company['Company']

            if data_flag[index-1] == 0 and ticker_flag[index-1] == 0:
                # Build dictionary  to access API
                data = { "function": function,  
                         "symbol": symbol,           
                         "apikey": api_key, 
                         "outputsize": outputsize} 

                # Read site and convert to JSON:
                page = requests.get(url, params = data).json()

                try:
                    # Parse the page: 
                    all_trades = parse_one_ticker(page)
                    all_trades['Name'] = firm_name

                    # Do the SQL import:
                    all_trades.to_sql(tablename, engine, if_exists='append', schema='stocks')

                    data_flag[index-1] = 1 # Set Flag
                    print(index, symbol, firm_name, 'Success')

                except KeyError:
                    pv = list(page.values())[0]
                    print(index, symbol, firm_name, pv)
                    if symbol not in failed_codes: 
                        failed_codes.append(symbol)
                        failed_msg.append(pv)
                    time.sleep(delay)

                    # If invalid API store as a flag.
                    if pv[:11] == 'Invalid API':
                        ticker_flag[index - 1] = 1               

        nlast, nthis = nthis, sum(data_flag)
        print('='*60)
        print('{} Companies found'.format(nthis))
        print('='*60)
        
    failed_df = pd.DataFrame(list(zip(failed_codes,failed_msg)))
        
    return failed_df 

In [33]:
# Define parameters for one function and then run it:
companies_df = FTSE100.iloc[:10]
tablename = 'Adjusted_all_FTSE'
schema = 'stocks'
api_key = "LSJQLCCGC6A3RJGC"    
function = "TIME_SERIES_DAILY_ADJUSTED"  # "TIME_SERIES_DAILY" "TIME_SERIES_DAILY_ADJUSTED
outputsize = 'full'  # 'full' 'compact'

vantage_scrape_list(companies_df, tablename, schema, engine, function, outputsize, api_key)

1 III 3i Success
2 ADM Admiral Group Success
3 AAL Anglo American plc Success
4 ANTO Antofagasta Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.
5 AHT Ashtead Group Success
6 ABF Associated British Foods Success
7 AZN AstraZeneca Success
8 AV Aviva Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.
9 BA BAE Systems Success
10 BARC Barclays Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.
7 Companies found
7 Companies found


,0,1
0,ANTO,Invalid API call. Please retry or visit the do...
1,AV,Invalid API call. Please retry or visit the do...
2,BARC,Invalid API call. Please retry or visit the do...


# Test Morningstar API: 

In [40]:
import pandas_datareader as pdr
from datetime import datetime 

today = datetime.now().strftime('%d-%m-%y')
start = '01-01-2015'

In [ ]:
fn = pdr.mstar.daily.MorningstarDailyReader('ANTO', start=start, end=today, incl_splits=True , incl_dividends=True)
company = fn.read()

In [44]:
company

Close     High     Low     Open   Volume    isDividend
Symbol Date                                                                
AV     2015-01-01  14.9000  14.9000  14.900  14.9000        0           NaN
       2015-01-02  14.8600  14.9800  14.820  14.9400    99391           NaN
       2015-01-05  14.3500  14.5700  14.270  14.5600   153991           NaN
       2015-01-06  14.0600  14.2800  13.970  14.2800   160004           NaN
       2015-01-07  14.0500  14.0700  13.940  14.0500   164367           NaN
       2015-01-08  14.5700  14.6400  14.400  14.4100   134682           NaN
       2015-01-09  14.8600  14.9899  14.790  14.9600   191464           NaN
       2015-01-12  14.7000  14.7700  14.590  14.7600   382786           NaN
       2015-01-13  14.8500  14.9401  14.730  14.7700   304402           NaN
       2015-01-14  15.0500  15.0900  14.910  14.9400   291203           NaN
       2015-01-15  15.3600  15.4500  15.140  15.2200   242818           NaN
       2015-01-16  15.5700  15.5700  15.290  15.4000  1348000           NaN
       2015-01-19  15.5700  15.5700  15.570  15.5700        0           NaN
       2015-01-20  15.8500  15.8600  15.730  15.7400  1056084           NaN
       2015-01-21  16.1500  16.1900  15.930  15.9600  1118609           NaN
       2015-01-22  16.3700  16.3800  16.220  16.2700   333903           NaN
       2015-01-23  16.1300  16.3000  16.110  16.2600   206916           NaN
       2015-01-26  16.4500  16.5200  16.310  16.3400   167815           NaN
       2015-01-27  16.3000  16.3800  16.190  16.2500   106250           NaN
       2015-01-28  16.0200  16.2900  16.010  16.1600   169168           NaN
       2015-01-29  16.3200  16.3500  16.110  16.1800   171432           NaN
       2015-01-30  15.8900  16.1300  15.890  16.0900   151924           NaN
       2015-02-02  16.0300  16.0500  15.890  15.9600   241761           NaN
       2015-02-03  16.3700  16.3900  16.140  16.1600   160470           NaN
       2015-02-04  16.1300  16.2550  16.070  16.1300   306773           NaN
       2015-02-05  16.3400  16.3700  16.240  16.2500    77658           NaN
       2015-02-06  16.3700  16.5700  16.350  16.5200   124293           NaN
       2015-02-09  16.2200  16.3300  16.200  16.2600   106077           NaN
       2015-02-10  16.5100  16.5600  16.420  16.4900   217414           NaN
       2015-02-11  16.3100  16.3600  16.220  16.3500   140508           NaN
...                    ...      ...     ...      ...      ...           ...
       2018-04-05  13.9600  13.9700  13.850  13.8500    45731  s:0.5256<br>
       2018-04-06  13.9500  14.3400  13.950  14.0100    40500           NaN
       2018-04-09  14.2900  14.3500  14.150  14.1500    82180           NaN
       2018-04-10  14.2800  14.4400  14.210  14.4250  1350123           NaN
       2018-04-11  14.2500  14.4300  14.200  14.3700  2373650           NaN
       2018-04-12  14.5350  14.6000  14.460  14.4600    42638           NaN
       2018-04-13  14.5100  14.5960  14.480  14.5700    42512           NaN
       2018-04-16  14.6000  14.6200  14.520  14.5200    86337           NaN
       2018-04-17  14.6800  14.6950  14.590  14.6200    84505           NaN
       2018-04-18  14.7950  14.9000  14.770  14.7800    63853           NaN
       2018-04-19  14.9500  15.0400  14.720  14.9800    70402           NaN
       2018-04-20  14.7150  14.8800  14.670  14.7400    60023           NaN
       2018-04-23  14.7700  15.0500  14.760  14.7700    91433           NaN
       2018-04-24  14.6400  14.8300  14.600  14.8100    88227           NaN
       2018-04-25  14.6600  14.8400  14.530  14.5300    30290           NaN
       2018-04-26  14.7200  14.8400  14.660  14.6700    93979           NaN
       2018-04-27  14.5800  14.7425  14.470  14.5000    85573           NaN
       2018-04-30  14.5900  14.6700  14.540  14.5600    48932           NaN
       2018-05-01  14.7300  14.7300  14.600  14.6500    23228           NaN
       2018-05-02  14.5800  14.7400  14.580  14.7100    83443        

In [1]:
c_list =''' 
III
ADM
AAL
ANTO
AHT
ABF
AZN
AV.
BA.
BARC
BDEV
BKG
BLT
BP.
BATS
BLND
BT.A
BNZL
BRBY
CCL
CNA
CCH
CPG
CRH
CRDA
DCC
DGE
DLG
EZJ
EVR
EXPN
FERG
FRES
GFS
GSK
GLEN
HLMA
HL.
HSBA
IMB
INF
IHG
IAG
ITRK
ITV
JMAT
JE.
KGF
LAND
LGEN
LLOY
LSE
MKS
MDC
MRO
MCRO
MNDI
MRW
NG.
NXT
NMC
OML
PPB
PSON
PSN
PRU
RRS
RB.
REL
RTO
RIO
RR.
RBS
RDSA
RDSB
RMG
RSA
SGE
SBRY
SDR
SMT
SGRO
SVT
SHP
SKY
SN.
SMDS
SMIN
SKG
SSE
STJ
STAN
SLA
TW.
TSCO
TUI
ULVR
UU.
VOD
WTB
WPP'''
c_list


tickers = c_list.split('\n')[1:]

tickersc= list(map(lambda x: x.replace('.',''),tickers))

In [3]:
company = []
for i, tick in enumerate(tickers[:10]):  
    print(i, tick)
    fn = pdr.mstar.daily.MorningstarDailyReader(tick, start=start, end=today, incl_splits=True , incl_dividends=True)
    company.append(fn.read())
    
# Doesn't work 
len(company)

0 III


NameError: name 'pdr' is not defined

In [6]:
pdr.mstar.daily

<module 'pandas_datareader.mstar.daily' from 'C:\\Users\\Student\\Anaconda3\\lib\\site-packages\\pandas_datareader\\mstar\\daily.py'>

In [ ]:
fn = pdr.mstar.daily.MorningstarDailyReader(tickers[:5], start=start, end=today, incl_splits=True , incl_dividends=True)
fn.read()

In [ ]:
# Throw a whole data frame in at a time into SQL
# Get a reliable list of all the companies.


In [47]:
for Comp in FTSE100:
    print(Comp)

Company
Ticker
FTSE sector[9]


In [58]:
for i, row in FTSE100.iterrows():
    print(row['Ticker'])

III
ADM
AAL
ANTO
AHT
ABF
AZN
AV.
BA.
BARC
BDEV
BKG
BLT
BP.
BATS
BLND
BT.A
BNZL
BRBY
CCL
CNA
CCH
CPG
CRH
CRDA
DCC
DGE
DLG
EZJ
EVR
EXPN
FERG
FRES
GFS
GSK
GLEN
HLMA
HL.
HSBA
IMB
INF
IHG
IAG
ITRK
ITV
JMAT
JE.
KGF
LAND
LGEN
LLOY
LSE
MKS
MDC
MRO
MCRO
MNDI
MRW
NG.
NXT
NMC
OML
PPB
PSON
PSN
PRU
RRS
RB.
REL
RTO
RIO
RR.
RBS
RDSA
RMG
RSA
SGE
SBRY
SDR
SMT
SGRO
SVT
SHP
SKY
SN.
SMDS
SMIN
SKG
SSE
STAN
SLA
STJ
TW.
TSCO
TUI
ULVR
UU.
VOD
WTB
WPP


In [90]:
df4 read.json(page)

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events',
  '2. Symbol': 'MSFT',
  '3. Last Refreshed': '2018-05-15',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2017-12-20': {'1. open': '86.2000',
   '2. high': '86.3000',
   '3. low': '84.7100',
   '4. close': '85.5200',
   '5. adjusted close': '85.1263',
   '6. volume': '23425009',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0000'},
  '2017-12-21': {'1. open': '86.0500',
   '2. high': '86.1000',
   '3. low': '85.4000',
   '4. close': '85.5000',
   '5. adjusted close': '85.1064',
   '6. volume': '16638402',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0000'},
  '2017-12-22': {'1. open': '85.4000',
   '2. high': '85.6300',
   '3. low': '84.9200',
   '4. close': '85.5100',
   '5. adjusted close': '85.1163',
   '6. volume': '14033977',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0000'},
  '2017-12-26': {